

# Visualizing Networks, and Why You Shouldn't

We've drawn networks several times in this course so far. Here's one of our favorite running examples, the Karate Club graph. Here's two ways to draw it. On the right, we've drawn the graph by assigning every node to a location in the unit square $[0,1]^2$ uniformly at random. On the right, we have used the default layout algorithm in NetworkX, which is invoked automatically when we call `nx.draw`.

In [ ]:
import networkx as nx
import numpy as np 
from matplotlib import pyplot as plt 

fig, ax = plt.subplots(1, 2, figsize = (8, 4))

def unweight(G):
    for source, target in G.edges():
        G[source][target]['weight'] = 1
    return G

G = unweight(nx.karate_club_graph())

draw_kwargs = {"node_color": "lightblue", "node_size": 100, "edge_color": "gray"}

nx.draw(G, ax = ax[1], **draw_kwargs)

def random_layout(G):
    return np.random.rand(G.number_of_nodes(), 2)

random_layout(G)

nx.draw(G, pos = random_layout(G), ax = ax[0], **draw_kwargs)

Which of these drawings look better to you? Which do you feel better helps you understand the structure of the graph? 

[![](https://www.data-imaginist.com/assets/img/edge_meme_wide.jpg) *Image credit: [Thomas Lin Pedersen](https://www.data-imaginist.com/posts/2017-02-16-ggraph-introduction-edges/)*]{.column-margin}
In this set of notes, we'll develop from scratch an algorithm for producing reasonably attractive visualizations of networks. We'll then discuss some of the significant limitations associated with drawing networks as points connected by line segments, and consider some alternatives. 

## Network Drawing as Isometry Approximation

We're going to view network drawing primarily as the problem of *placing the nodes*. This corresponds to the following problem: 
[For drawing on a computer screen or chalkboard, $d = 2$. The general task of placing nodes in Euclidean space is often called *graph embedding*, and has applications in deep learning for much higher values of $d$.]{.aside}

> To each node $i \in N$, assign a vector $\mathbf{x}_i \in \mathbb{R}^d$. 

If we collect all the nodes together, we aim to find a *matrix* $\mathbf{X} \in \mathbb{R}^{n \times d}$, where $n$ is the number of nodes in the graph, such that the $i$th row of $\mathbf{X}$ is the vector $\mathbf{x}_i$ assigned to node $i$.

Of course, we don't want to just find *any* old matrix $\mathbf{X}$; that's what we did on the random visualization above and the result was not very impressive. Instead, we want to find a matrix $\mathbf{X}$ that reflects the structure of the graph in some useful way. There are multiple ways to 

There are multiple ways to approach this problem, but in these notes, we are going to follow the approach of @kamada1989algorithm. Their idea is simple to state: 

> Choose $\mathbf{X}$ so that the Euclidean distance $d^e_{ij} = \lVert \mathbf{x}_i - \mathbf{x}_j \rVert$ is approximately the same as the graph geodesic distance $d^g_{ij}$ between nodes $i$ and $j$, for all pairs of nodes $(i,j)$. 

This approach is an expression of the geometric idea of *isometry*: we are looking to map the nodes from one metric space (the graph) into another metric space (Euclidean space) in a way that approximately preserves distance.

To make this actionable, let's define an optimization objective that measures how far apart the Euclidean distances are from the graph geodesic distances: 

$$
\begin{aligned}
    f(\mathbf{X}, G) &= \sum_{(i,j) \in \binom{N}{2}} \left( d^e_{ij} - d^g_{ij} \right)^2    \\ 
                     &= \sum_{(i,j) \in \binom{N}{2}} \left( \lVert \mathbf{x}_i - \mathbf{x}_j \rVert - d^g_{ij} \right)^2\;.
\end{aligned}
$$

This objective function is also sometimes called an "energy" of the problem. 

The calculation of $d^g_{ij}$ for each pair $i$ and $j$ can be done at the beginning of the optimization process and then stored. [These distances are expensive to both compute and store, but (as we'll discuss more below), graph drawing for larger graphs is not recommended in any case.]{.aside} Having computed the distances, we need to pick the positions of the nodes in order to minimize the objective $f(\mathbf{X}, G)$. Unfortunately there is no way to do this in closed form, and so we need to use iterative methods like gradient descent. The gradient of $f$ with respect to the coordinates $\mathbf{x}_{k1} = (x_{k1}, x_{k2},\ldots)$ of a single node $k$ is 

$$
\begin{aligned}
    \nabla_{\mathbf{x}_k} f(\mathbf{X}, G) &= 2 \sum_{j \in N\setminus k} \left( \lVert \mathbf{x}_k - \mathbf{x}_j \rVert - d^g_{kj} \right) \frac{\mathbf{x}_k - \mathbf{x}_j}{\lVert \mathbf{x}_k - \mathbf{x}_j \rVert} \;.
\end{aligned}
$$

In gradient descent, we update the position of each node $k$ by moving it in the direction of the negative gradient: $\mathbf{x}_k \gets \mathbf{x}_k - \alpha \nabla_{\mathbf{x}_k} f(\mathbf{X}, G)$, where $\alpha$ is a small positive number that in this age of machine learning is often called the *learning rate*.

Here's an implementation of the gradient: 

We'll follow the original version of this algorithm, which cycles through the nodes, computes this gradient, and updates each node one at a time. Here's an implementation. We'll pass the distance matrix $D$ to the function `grad_f` to avoid recomputing it at each step. 

How does it look? 

In [ ]:
# draw the result!
nx.draw(G, pos = {i: X[i] for i in range(G.number_of_nodes())}, **draw_kwargs)

Our result looks pretty reasonable! Nodes that "should" be close to each other do indeed appear to be drawn close together in the graph. 

The results are not always reliable; if we check the Les Miserables graph, we might argue that a lot of the nodes in the dense center of the graph are a little too squished together: 

In [ ]:
G = nx.convert_node_labels_to_integers(nx.les_miserables_graph())
X = optimization_layout(G, grad_fun = grad_f, alpha = 0.01, n_iter = 100)
nx.draw(G, pos = {i: X[i] for i in range(G.number_of_nodes())}, **draw_kwargs)

### An Adjustment

We can compensate for this by weighting pairs of nodes differently. In the final form of the Kamada-Kawai algorithm, we modify the objective function by incorporating an inverse-square distance weighting. The modified objective function is 

$$
\begin{aligned}
    f(\mathbf{X}, G)  &= \sum_{(i,j) \in \binom{N}{2}} \frac{1}{\left(d^g_{ij}\right)^2}\left( \lVert \mathbf{x}_i - \mathbf{x}_j \rVert - d^g_{ij} \right)^2\;.
\end{aligned}
$$

and the gradient with respect to a single node position is 

$$
\begin{aligned}
    \nabla_{\mathbf{x}_k} f(\mathbf{X}, G) &= 2 \sum_{j \in N\setminus k} \frac{1}{\left(d^g_{kj}\right)^2}\left( \lVert \mathbf{x}_k - \mathbf{x}_j \rVert - d^g_{kj} \right) \frac{\mathbf{x}_k - \mathbf{x}_j}{\lVert \mathbf{x}_k - \mathbf{x}_j \rVert} \;.
\end{aligned}
$$

Let's give this one a try: 

In [ ]:
def grad_f_weighted(X, D, k):
    grad = np.zeros(X.shape[1])
    for j in range(X.shape[0]):
        if j == k:
            continue
        grad += (np.linalg.norm(X[k] - X[j]) - D[k, j]) * (X[k] - X[j]) / np.linalg.norm(X[k] - X[j]) / D[k,j]**2
    return 2 * grad

X = optimization_layout(G, grad_fun = grad_f_weighted, alpha = 0.01, n_iter = 1000)

nx.draw(G, pos = {i: X[i] for i in range(G.number_of_nodes())}, **draw_kwargs)

There are many other ways to draw networks, several of which are implemented as built-in methods in NetworkX. See the [documentation](https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout) for various other possibilities. 

## Why You Shouldn't Draw Networks

Making attractive visualizations of large networks is a very fun and satisfying thing to do, and some software packages like [Gephi](https://gephi.org/) are specifically designed for this task. We encourage you to put network visualizations on your phone wallpaper, t-shirts, posters, websites, etc. [Indeed, Phil Chodrow, one of the coauthors of these notes, made a graph visualization as the image background of [his website](https://www.philchodrow.prof/).]

So, what do we mean when we say that you shouldn't draw networks? In general, it's very difficult to extract reliable structural insights about networks by eye. This means that the one place drawings of networks *usually* don't belong is in scientific papers. It's just too easy for the eye to be drawn to structural features that may or may not actually be present in the data. For this reason, node-edge visualizations of large networks have been called names like "[ridiculograms](https://petterhol.me/2018/05/28/ridiculograms-a-ridiculous-dialogue/)" and [hairballs](https://skewed.de/tiago/posts/hairball/) by prominent network scientists. 

## What You Should Do Instead

The right way to visualize the structure of your network is *very* context-dependent, and there are many, many possibilities. Here we'll point out just one: the adjacency matrix. This is a common strategy for visualizing networks that separate into one or more distinct clusters, sometimes also called "communities." For example, simply inspecting the adjacency matrix of the Les Miserables graph can reveal a lot about its structure: 

In [ ]:
G = nx.les_miserables_graph()
G = unweight(G)
G = nx.convert_node_labels_to_integers(G)
A = nx.to_numpy_array(G)
plt.imshow(A, cmap = "Greys", vmax = 1.2)

The adjacency matrix allows to easily see the presence of dense clusters (indeed, several cliques) in the graph, as well as a few nodes who seem to interact with almost all of the other ones. 

In this case, the adjacency matrix was already sorted by node in a way that made the structure clear. In more complicated cases, we may need to use a *community detection algorithm* to find a way to sort the nodes that reveals useful structure. This is a complicated (and thorny) topic which we'll touch on later in this course. 

## References